In [1]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
from pathlib import Path
import csv
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
# import lightgbm as lgb

from sklearn.metrics import (precision_score, recall_score, roc_auc_score, accuracy_score, mean_squared_error,
                             confusion_matrix, precision_recall_curve, roc_curve, brier_score_loss)

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [ ]:
columns_drop = {
    "Facebook_data": None,
    "Features_TestSet": None,
    "House_Price_Adv_Regression": ['Id'],
    "Instant_Liking": None,
    "Insurance": None,
    "Isolet": None,
    "new_data_trans": None,
    "OnlineNewsPopularity": None,
    "ParkinsonData": ['subject.'],
    "Sberbank_Russian_Housing_Market": ['id'],
    "slice_localization_data": None,
    "Telecom_data": ['Phone.Number'],
    "yearMSD_new": None,
    "arrhythmia": None,
    "Big_mart_sales": ['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'],
    "blogData": None,
    "communities": ['x4', 'x1'],
    "dengue_features": ['year', 'weekofyear', 'week_start_date'],
    "ECG0_p02": None,
    "ENERGY_DATA_COMPLETE": None
}

fill_na = {
    "Facebook_data": False,
    "Features_TestSet": False,
    "House_Price_Adv_Regression": False,
    "Instant_Liking": False,
    "Insurance": False,
    "Isolet": False,
    "new_data_trans": False,
    "OnlineNewsPopularity": False,
    "ParkinsonData": False,
    "Sberbank_Russian_Housing_Market": False,
    "slice_localization_data": False,
    "Telecom_data": False,
    "yearMSD_new": False,
    "arrhythmia": False,
    "Big_mart_sales": True,
    "blogData": False,
    "communities": False,
    "dengue_features": False,
    "ECG0_p02": False,
    "ENERGY_DATA_COMPLETE": False
}

fill_na_column = {
    "Facebook_data": [],
    "Features_TestSet": [],
    "House_Price_Adv_Regression": [],
    "Instant_Liking": [],
    "Insurance": [],
    "Isolet": [],
    "new_data_trans": [],
    "OnlineNewsPopularity": [],
    "ParkinsonData": [],
    "Sberbank_Russian_Housing_Market": [],
    "slice_localization_data": [],
    "Telecom_data": [],
    "yearMSD_new": [],
    "arrhythmia": [],
    "Big_mart_sales": ['Outlet_Size'],
    "blogData": [],
    "communities": [],
    "dengue_features": [],
    "ECG0_p02": [],
    "ENERGY_DATA_COMPLETE": [],
}

def process_dataset(df_train, df_test, columns_drop, fill_na, fill_na_columns):    
    df_total = pd.concat([df_train, df_test], axis=0)
    tar = df_total.iloc[:, -1]
    df_total = df_total.iloc[:, :-1]
    
    if fill_na:
        for column in fill_na_columns:
            df_total[column].fillna("NOT_PRESENT", inplace=True)
    
    if columns_drop:
        df_total = df_total.drop(columns_drop, axis='columns')
    
    cat = df_total.select_dtypes(include=['object']).columns.to_list()
    df_total = pd.get_dummies(df_total, cat)
    
    df_total = pd.concat([df_total, tar], axis=1)
    df_train = df_total.iloc[:df_train.shape[0],:]
    df_test = df_total.iloc[df_train.shape[0]:,:]
    
    return df_train, df_test

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    # "Facebook_data",
    # "Features_TestSet",
    # "House_Price_Adv_Regression",
    # "Instant_Liking",
    # "Insurance",
    "Isolet",
    # "new_data_trans",
    # "OnlineNewsPopularity",
    # "ParkinsonData",
    # "Sberbank_Russian_Housing_Market",
    # "slice_localization_data",
    # "Telecom_data",
    # "yearMSD_new",
    # "arrhythmia",
    # "Big_mart_sales",
    # "blogData",
    # "communities",
    # "dengue_features",
    # "ECG0_p02",
    # "ENERGY_DATA_COMPLETE"
]

# Fill missing for Instant_Liking

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

for index, dataset in enumerate(datasets):
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(seed) + '.csv')
        df_train = pd.read_csv(csv_path_train)
        
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '.csv')
        df_test = pd.read_csv(csv_path_test)
        
        df_train, df_test = process_dataset(df_train, df_test, columns_drop[dataset], fill_na[dataset], fill_na_column[dataset])
        df_train.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(seed) + '_modified.csv'), index=False)
        df_test.to_csv(os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv'), index=False)

In [ ]:
# Specifically for big mart dataset
def process_big_mart(df_train):
    df_train['Outlet_Size'].fillna("NOT_PRESENT", inplace=True)
    
    df_train = df_train.drop(['Item_Identifier', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales'], axis='columns')
    
    dummies = pd.get_dummies(df_train.Item_Fat_Content)
    df_train = pd.concat([df_train, dummies], axis='columns')
    dummies = pd.get_dummies(df_train.Item_Type)
    df_train = pd.concat([df_train, dummies], axis='columns')
    dummies = pd.get_dummies(df_train.Outlet_Size)
    df_train = pd.concat([df_train, dummies], axis='columns')
    dummies = pd.get_dummies(df_train.Outlet_Location_Type)
    df_train = pd.concat([df_train, dummies], axis='columns')
    dummies = pd.get_dummies(df_train.Outlet_Type)
    df_train = pd.concat([df_train, dummies], axis='columns')
    df_train['Item_Outlet_Sales_Log'] = np.log(df_train['Item_Outlet_Sales'])
    return df_train

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)



data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
dataset = "Big_mart_sales"

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]
for s in tqdm(seeds):
    csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '.csv')
    df_train = pd.read_csv(csv_path_train)
    csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '.csv')
    df_test = pd.read_csv(csv_path_test)
    
    df_train = process_big_mart(df_train)
    df_test = process_big_mart(df_test)
    df_train.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '_modified.csv'), index=False)
    df_test.to_csv(os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '_modified.csv'), index=False)
    

In [ ]:
def RF_pred_kfold(X, y, details=None):
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=RandomForestRegressor(),
        param_distributions={
            'n_estimators': range(1, 200, 10),
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
    print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)

    # Build models with optimized hyperparameters
    model_RF = RandomForestRegressor(
        n_estimators=search_result.best_params_["n_estimators"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"]
    )
    

    # Split dataset into 5 consecutive folds
    kf = KFold(n_splits=5, shuffle=True, random_state=None)
    
    i = 1
    for train, test in kf.split(X):
        X_train = X.iloc[train,:]
        y_train = y.iloc[train,:]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        model_RF.fit(X_train, y_train.values.ravel())
        train_pred = model_RF.predict(X_train)
        y_pred = model_RF.predict(X_test)
        
#         train_accuracy = accuracy_score(y_train, train_pred)
#         train_precision = precision_score(y_train, train_pred)
#         train_recall = recall_score(y_train, train_pred)
#         train_auc = roc_auc_score(y_train, train_pred)
        train_rmse = mean_squared_error(y_train, train_pred, squared=False)
        
#         test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
        test_rmse = mean_squared_error(y_test, y_pred, squared=False)
        
        details['fold'] = str(i)
        details['training_rmse'] = train_rmse
        details['testing_rmse'] = test_rmse
        
        print('Fold '+ str(i), ':  Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)
        
        i += 1

    return model_RF

In [ ]:
# Decision Tree module
def DT_pred(train_path, validation_path, train_column_drop, validation_column, details=None):
    
    df_train = pd.read_csv(train_path)
    df_validation = pd.read_csv(validation_path)
    
    X = df_train.drop(train_column_drop, axis = 1)
    y = df_train[validation_column]
    
    X_validation = df_validation.drop(train_column_drop, axis = 1)
    y_validation = df_validation[validation_column]
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=DecisionTreeRegressor(),
        param_distributions={
            'criterion': ['squared_error'],
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, 
        cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose = True)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
    print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)

    # Build models with optimized hyperparameters
    model_DT = DecisionTreeRegressor(
        criterion=search_result.best_params_["criterion"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"])

    model_DT.fit(X, y.values.ravel())
    train_pred = model_DT.predict(X)
    y_pred = model_DT.predict(X_validation)

#         train_accuracy = accuracy_score(y_train, train_pred)
#         train_precision = precision_score(y_train, train_pred)
#         train_recall = recall_score(y_train, train_pred)
#         train_auc = roc_auc_score(y_train, train_pred)
    train_rmse = mean_squared_error(y, train_pred, squared=False)

#         test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
    test_rmse = mean_squared_error(y_validation, y_pred, squared=False)

    details['training_rmse'] = train_rmse
    details['testing_rmse'] = test_rmse

    print('Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)

    return model_DT

In [ ]:
# Random Forest Module
def RF_pred(train_path, validation_path, train_column_drop, validation_column, details=None):
    
    df_train = pd.read_csv(train_path)
    df_validation = pd.read_csv(validation_path)
    
    X = df_train.drop(train_column_drop, axis = 1)
    y = df_train[validation_column]
    
    X_validation = df_validation.drop(train_column_drop, axis = 1)
    y_validation = df_validation[validation_column]
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=RandomForestRegressor(),
        param_distributions={
            'n_estimators': range(1, 200, 10),
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
    print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)

    # Build models with optimized hyperparameters
    model_RF = RandomForestRegressor(
        n_estimators=search_result.best_params_["n_estimators"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"]
    )
    
    # for train, test in kf.split(X):
    model_RF.fit(X, y.values.ravel())
    train_pred = model_RF.predict(X)
    y_pred = model_RF.predict(X_validation)

#         train_accuracy = accuracy_score(y_train, train_pred)
#         train_precision = precision_score(y_train, train_pred)
#         train_recall = recall_score(y_train, train_pred)
#         train_auc = roc_auc_score(y_train, train_pred)
    train_rmse = mean_squared_error(y, train_pred, squared=False)

#         test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
    test_rmse = mean_squared_error(y_validation, y_pred, squared=False)

    details['training_rmse'] = train_rmse
    details['testing_rmse'] = test_rmse

    print('Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)

    return model_RF

In [2]:
# Gradient Boosting Module
def GB_pred(train_path, validation_path, train_column_drop, validation_column, details=None):
    
    df_train = pd.read_csv(train_path)
    df_validation = pd.read_csv(validation_path)
    
    X = df_train.drop(train_column_drop, axis = 1)
    y = df_train[validation_column]
    
    X_validation = df_validation.drop(train_column_drop, axis = 1)
    y_validation = df_validation[validation_column]
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=GradientBoostingRegressor(loss='squared_error'),
        param_distributions={
            'n_estimators': range(1, 200, 10),
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']
        },
        cv=5,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
    print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)

    # Build models with optimized hyperparameters
    model_GB = GradientBoostingRegressor(
        n_estimators=search_result.best_params_["n_estimators"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"],
        loss='squared_error'
    )
    
    # for train, test in kf.split(X):
    model_GB.fit(X, y.values.ravel())
    train_pred = model_GB.predict(X)
    y_pred = model_GB.predict(X_validation)

#         train_accuracy = accuracy_score(y_train, train_pred)
#         train_precision = precision_score(y_train, train_pred)
#         train_recall = recall_score(y_train, train_pred)
#         train_auc = roc_auc_score(y_train, train_pred)
    train_rmse = mean_squared_error(y, train_pred, squared=False)

#         test_accuracy = accuracy_score(y_test, y_pred)
#         test_precision = precision_score(y_test, y_pred)
#         test_recall = recall_score(y_test, y_pred)
#         test_auc = roc_auc_score(y_test, y_pred)
    test_rmse = mean_squared_error(y_validation, y_pred, squared=False)

    details['training_rmse'] = train_rmse
    details['testing_rmse'] = test_rmse

    print('Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
#         print('Fold '+ str(i), ':  Training precision: ', train_precision, 'Testing precision: ', test_precision)
#         print('Fold '+ str(i), ':  Training recall: ', train_recall, 'Testing accuracy: ', test_recall)
#         print('Fold '+ str(i), ':  Training auc: ', train_auc, 'Testing auc: ', test_auc)

    return model_GB

In [3]:
# normal Random Forest and Decision Tree with train set and validation set

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    # "Facebook_data", # -> done,
    # "Features_TestSet", # -> done,
    # "House_Price_Adv_Regression", # -> done,
    # "Instant_Liking",
    # "Insurance", # -> done,
    # "Isolet",
    # "new_data_trans", # -> done,
    # "OnlineNewsPopularity", # -> done,
    # "ParkinsonData", # -> done,
    # "Sberbank_Russian_Housing_Market", # -> done, Gradiant Boosting remains
    # "slice_localization_data", # -> done,
    # "Telecom_data", # -> done,
    # "yearMSD_new", # -> done,
    # "arrhythmia", # -> done,
    # "Big_mart_sales", # -> done,
    # "blogData", # -> done
    # "communities", # -> done,
    # "dengue_features", # -> done,
    # "ECG0_p02", # -> done,
    # "ENERGY_DATA_COMPLETE", # -> done
]

x_column_drop = {
    "Facebook_data": ['Unnamed: 0', 'Total.Interactions'],
    "Features_TestSet": ['Unnamed: 0', 'Target'],
    "House_Price_Adv_Regression": ['Unnamed: 0', 'SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    "Insurance": ['Unnamed: 0', 'charges'],
    "Isolet": ['Unnamed: 0', 'Target'], # -> some random error, need to see,
    "new_data_trans": ['Unnamed: 0', 'X23.Humedad_Exterior_Sensor'],
    "OnlineNewsPopularity": ['Unnamed: 0', 'shares'],
    "ParkinsonData": ['Unnamed: 0', 'total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['Unnamed: 0', 'price_doc'],
    "slice_localization_data": ['Unnamed: 0', 'reference'],
    "Telecom_data": ['Unnamed: 0', 'Churned.Label'],
    "yearMSD_new": ['Unnamed: 0', 'Year'],
    "arrhythmia": ['Unnamed: 0', 'Defection'],
    "Big_mart_sales": ['Unnamed: 0', 'Item_Outlet_Sales'],
    "blogData": ['Unnamed: 0', 'Comments'],
    "communities": ['Unnamed: 0','ViolentCrimesPerPop'],
    "dengue_features": ['Unnamed: 0','total_cases'],
    "ECG0_p02": ['Unnamed: 0', 'CurrentValue'],
    "ENERGY_DATA_COMPLETE": ['Unnamed: 0', 'Appliances']
}

target_column = {
    "Facebook_data": ['Total.Interactions'],
    "Features_TestSet": ['Target'],
    "House_Price_Adv_Regression": ['SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    "Insurance": ['charges'],
    "Isolet": ['Target'], # -> some random error, need to see,
    "new_data_trans": ['X23.Humedad_Exterior_Sensor'],
    "OnlineNewsPopularity": ['shares'],
    "ParkinsonData": ['total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['price_doc'],
    "slice_localization_data": ['reference'],
    "Telecom_data": ['Churned.Label'],
    "yearMSD_new": ['Year'],
    "arrhythmia": ['Defection'],
    "Big_mart_sales": ['Item_Outlet_Sales'],
    "blogData": ['Comments'],
    "communities": ['ViolentCrimesPerPop'],
    "dengue_features": ['total_cases'],
    "ECG0_p02": ['CurrentValue'],
    "ENERGY_DATA_COMPLETE": ['Appliances']
}

# dataset = "Big_mart_sales"
seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

# For RF
# for index, dataset in enumerate(datasets):
#     df = pd.DataFrame()
#     print(dataset)
#     for seed in tqdm(seeds):
#         csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
#         csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
#         csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
#         df_test = pd.read_csv(csv_path_test)
        
#         # print(csv_path_train, csv_path_test)
        
#         X_test = df_test.drop(x_column_drop[dataset], axis = 1)
#         y_test = df_test[target_column[dataset]]
        
#         details = {
#             'dataset': dataset,
#             'seed': str(seed)
#         }
        
#         # For RF
#         model_RF = RF_pred(csv_path_train, csv_path_validation, x_column_drop[dataset], target_column[dataset], details)
        
#         importances = model_RF.feature_importances_
#         indices = np.argsort(importances)[::-1]
#         top_k = 10
#         top_indices = indices[:top_k]
#         details['best_feature_list'] = np.array(X_test.columns)[indices][0:top_k]
        
#         y_pred = model_RF.predict(X_test)
#         validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
#         details['validation_rmse'] = validation_rmse
        
#         df = df.append(details, ignore_index=True)
#         filepath = Path(dataset + '_RF.csv')
#         filepath.parent.mkdir(parents=True, exist_ok=True)
#         df.to_csv(filepath, index=False)

# For DT
# for index, dataset in enumerate(datasets):
#     df = pd.DataFrame()
#     print(dataset)
#     for seed in tqdm(seeds):
#         csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
#         csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
#         csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
#         df_test = pd.read_csv(csv_path_test)
        
#         # print(csv_path_train, csv_path_test)
        
#         X_test = df_test.drop(x_column_drop[dataset], axis = 1)
#         y_test = df_test[target_column[dataset]]
        
#         details = {
#             'dataset': dataset,
#             'seed': str(seed)
#         }
        
#         # For DT
#         model_DT = DT_pred(csv_path_train, csv_path_validation, x_column_drop[dataset], target_column[dataset], details)
        
#         importances = model_DT.feature_importances_
#         indices = np.argsort(importances)[::-1]
#         top_k = 10
#         top_indices = indices[:top_k]
#         details['best_feature_list'] = np.array(X_test.columns)[indices][0:top_k]
        
#         y_pred = model_DT.predict(X_test)
#         validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
#         details['validation_rmse'] = validation_rmse
        
#         df = df.append(details, ignore_index=True)
#         filepath = Path(dataset + '_DT.csv')
#         filepath.parent.mkdir(parents=True, exist_ok=True)
#         df.to_csv(filepath, index=False)

# For GB
for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
        csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
        df_test = pd.read_csv(csv_path_test)
        
        # print(csv_path_train, csv_path_test)
        
        X_test = df_test.drop(x_column_drop[dataset], axis = 1)
        y_test = df_test[target_column[dataset]]
        
        details = {
            'dataset': dataset,
            'seed': str(seed)
        }
        
        # For GB
        model_GB = GB_pred(csv_path_train, csv_path_validation, x_column_drop[dataset], target_column[dataset], details)
        
        importances = model_GB.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X_test.columns)[indices][0:top_k]
        
        y_pred = model_GB.predict(X_test)
        validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
        details['validation_rmse'] = validation_rmse
        
        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_GB.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)

slice_localization_data


  0%|                                                                                                                                                                                                   | 0/10 [00:00<?, ?it/s]

Best using:  {'n_estimators': 81, 'max_features': 'sqrt', 'max_depth': 41} Score:  -1.4605271859390634
Training RMSE:  0.004411825181726962 Testing RMSE:  1.3941839523525894


 10%|██████████████████▍                                                                                                                                                                     | 1/10 [11:44<1:45:41, 704.60s/it]/home/asim/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best using:  {'n_estimators': 171, 'max_features': 'sqrt', 'max_depth': 61} Score:  -1.3618377913222068
Training RMSE:  3.346092648416899e-07 Testing RMSE:  1.2655361691798528


...

Training RMSE:  0.46985066863732633 Testing RMSE:  72.07983695803388
Best using:  {'n_estimators': 191, 'max_features': 'sqrt', 'max_depth': 11} Score:  -73.58673744950609


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                  | 9/10 [21:34<02:09, 129.32s/it]

Training RMSE:  5.7506112577072654 Testing RMSE:  71.16385003943756
Best using:  {'n_estimators': 191, 'max_features': 'sqrt', 'max_depth': 31} Score:  -73.46909608921402


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [24:02<00:00, 144.24s/it]

Training RMSE:  2.132407021622937e-07 Testing RMSE:  76.62837236675169


In [ ]:
# normal Random Forest with k fold -> do not need this

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    # "Facebook_data",
    # "Features_TestSet",
    # "House_Price_Adv_Regression",
    # "Instant_Liking",
    # "Insurance",
    # "Isolet",
    # "new_data_trans",
    # "OnlineNewsPopularity",
    "ParkinsonData",
    # "Sberbank_Russian_Housing_Market",
    # "slice_localization_data",
    # "Telecom_data",
    # "yearMSD_new",
    # "arrhythmia",
    # "Big_mart_sales",
    # "blogData",
    # "communities",
    # "dengue_features",
    # "ECG0_p02",
    # "ENERGY_DATA_COMPLETE"
]

x_column_drop = {
    # "Facebook_data": ['Total.Interactions'],
    # "Features_TestSet": ['Unnamed: 0', 'Target'],
    # "House_Price_Adv_Regression": ['SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    # "Insurance": ['charges'],
    # "Isolet": ['Target'] -> some random error, need to see,
    # "new_data_trans": ['X23.Humedad_Exterior_Sensor'],
    # "OnlineNewsPopularity": ['shares'],
    "ParkinsonData": ['Unnamed: 0','total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['Unnamed: 0', 'price_doc'],
    "slice_localization_data": ['Unnamed: 0', 'reference'],
    # "Telecom_data": ['Churned.Label'],
    # "yearMSD_new": ['Year'],
    # "arrhythmia": ['Defection'],
    # "Big_mart_sales": ['Item_Outlet_Sales'],
    # "blogData": ['Comments'],
    # "communities": ['ViolentCrimesPerPop'],
    # "dengue_features": ['total_cases'],
    # "ECG0_p02": ['CurrentValue'],
    # "ENERGY_DATA_COMPLETE": ['Appliances']
}

target_column = {
    # "Facebook_data": ['Total.Interactions'],
    # "Features_TestSet": ['Target'],
    # "House_Price_Adv_Regression": ['SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    # "Insurance": ['charges'],
    # "Isolet": ['Target'] -> some random error, need to see,
    # "new_data_trans": ['X23.Humedad_Exterior_Sensor'],
    # "OnlineNewsPopularity": ['shares'],
    "ParkinsonData": ['total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['price_doc'],
    "slice_localization_data": ['reference'],
    # "Telecom_data": ['Churned.Label'],
    # "yearMSD_new": ['Year'],
    # "arrhythmia": ['Defection'],
    # "Big_mart_sales": ['Item_Outlet_Sales'],
    # "blogData": ['Comments'],
    # "communities": ['ViolentCrimesPerPop'],
    # "dengue_features": ['total_cases'],
    # "ECG0_p02": ['CurrentValue'],
    # "ENERGY_DATA_COMPLETE": ['Appliances']
}

# dataset = "Big_mart_sales"
seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(seed) + '_modified.csv')
        df_train = pd.read_csv(csv_path_train)
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
        df_test = pd.read_csv(csv_path_test)
        
        X = df_train.drop(x_column_drop[dataset], axis = 1)
        y = df_train[target_column[dataset]]
        X_test = df_test.drop(x_column_drop[dataset], axis = 1)
        y_test = df_test[target_column[dataset]]
        
        details = {
            'dataset': dataset,
            'seed': str(seed)
        }
        
        model_DT = DT_pred(X, y, details)
        model_RF = RF_pred_kfold(X, y, details)
        
        importances = model_RF.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X.columns)[indices][0:top_k]
        
        y_pred = model_RF.predict(X_test)
        validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
        details['validation_rmse'] = validation_rmse
        
        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_RF_kfold.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)

In [ ]:
# Split dataset into training and validation sets

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    # "Facebook_data",
    # "Features_TestSet",
    # "House_Price_Adv_Regression",
    # "Instant_Liking",
    # "Insurance",
    "Isolet",
    # "new_data_trans",
    # "OnlineNewsPopularity",
    # "ParkinsonData",
    # "Sberbank_Russian_Housing_Market",
    # "slice_localization_data",
    # "Telecom_data",
    # "yearMSD_new",
    # "arrhythmia",
    # "Big_mart_sales",
    # "blogData",
    # "communities",
    # "dengue_features",
    # "ECG0_p02",
    # "ENERGY_DATA_COMPLETE"
]

train_split = 0.8

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        data = []
        # print(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(seed) + '_modified.csv'))
        for each_line in open(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(seed) + '_modified.csv')):
            each_line = each_line.strip().split(',')
            data.append(each_line)
            
        headers = data.pop(0)
        
        random.shuffle(data)
        
        train_datapoints = int(train_split * len(data))
        train_set = [headers] + data[:train_datapoints]
        validation_set = [headers] + data[train_datapoints:]
        
        train_set_df = pd.DataFrame(train_set)
        train_set_df.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv'), index=False, header=False)
        
        validation_set_df = pd.DataFrame(validation_set)
        validation_set_df.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv'), index=False, header=False)
        

In [ ]:
# dataset normalization
from sklearn.preprocessing import MinMaxScaler

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    # "Facebook_data",
    # "Features_TestSet",
    # "House_Price_Adv_Regression",
    # "Instant_Liking",
    # "Insurance",
    # "Isolet",
    # "new_data_trans",
    # "OnlineNewsPopularity",
    "ParkinsonData",
    # "Sberbank_Russian_Housing_Market",
    # "slice_localization_data",
    # "Telecom_data",
    # "yearMSD_new",
    # "arrhythmia",
    # "Big_mart_sales",
    # "blogData",
    # "communities",
    # "dengue_features",
    # "ECG0_p02",
    # "ENERGY_DATA_COMPLETE"
]

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
        csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
        
        train_set = []
        for each_line in open(csv_path_train):
            each_line = each_line.strip().split(',')
            train_set.append(each_line)
        headers = train_set.pop(0)
        
        validation_set = []
        for each_line in open(csv_path_validation):
            each_line = each_line.strip().split(',')
            validation_set.append(each_line)
        validation_set.pop(0)
        
        test_set = []
        for each_line in open(csv_path_test):
            each_line = each_line.strip().split(',')
            test_set.append(each_line)
        test_set.pop(0)
        
        scaler = MinMaxScaler()
        train_set = [headers] + scaler.fit_transform(train_set).tolist()        
        validation_set = [headers] + scaler.transform(validation_set).tolist()
        test_set = [headers] + scaler.transform(test_set).tolist()
        
        train_set_df = pd.DataFrame(train_set)
        train_set_df.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train_normalized.csv'), index=False, header=False)
        
        validation_set_df = pd.DataFrame(validation_set)
        validation_set_df.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation_normalized.csv'), index=False, header=False)
        
        test_set_df = pd.DataFrame(test_set)
        test_set_df.to_csv(os.path.join(data_dir, dataset, "Test", dataset + '_seed_' + str(seed) + '_test_normalized.csv'), index=False, header=False)

In [ ]:
data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
dataset = "arrhythmia"
seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

for s in tqdm(seeds):
    csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '_modified.csv')
    df_train = pd.read_csv(csv_path_train)
    csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '_modified.csv')
    df_test = pd.read_csv(csv_path_test)
    X = df_train.drop(['Defection'], axis = 1)
    y = df_train[['Defection']]
    model_DT = DT_pred(X, y)
    
    model_RF = RF_pred(X, y)
    importances = model_RF.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_k = 10
    top_indices = indices[:top_k]
    
    print("Feature_importance: \n")
    print(np.array(X.columns)[indices][0:top_k])
    print(importances[top_indices])


In [ ]:
# Just need to drop one column and replace with a one-hot encoding

def process_big_mart(df_train):
    dummies = pd.get_dummies(df_train.Type)
    df_train = pd.concat([df_train, dummies], axis='columns')
    df_train = df_train.drop(['Type'], axis='columns')
    return df_train

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)



data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
dataset = "Facebook_data"

seeds = [50, 100, 150, 200, 250, 300, 350, 400, 450]
for s in tqdm(seeds):
    csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '.csv')
    df_train = pd.read_csv(csv_path_train)
    csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '.csv')
    df_test = pd.read_csv(csv_path_test)
    
    df_train = process_big_mart(df_train)
    df_test = process_big_mart(df_test)
    df_train.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '_modified.csv'), index=False)
    df_test.to_csv(os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '_modified.csv'), index=False)
    

In [ ]:
# No preprocessing required here

def process_big_mart(df_train):
    return df_train

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)



data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
dataset = "Features_TestSet"

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]
for s in tqdm(seeds):
    csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '.csv')
    df_train = pd.read_csv(csv_path_train)
    csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '.csv')
    df_test = pd.read_csv(csv_path_test)
    
    df_train = process_big_mart(df_train)
    df_test = process_big_mart(df_test)
    df_train.to_csv(os.path.join(data_dir, dataset, "Train", dataset + '_Train_seed' + str(s) + '_modified.csv'), index=False)
    df_test.to_csv(os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(s) + '_modified.csv'), index=False)